# VpCore2 3D analyse coordinate validation

Generate video, and check analyze match with original generate coordinate.

We use Cartesian distance calculation:
\begin{align}
d = \sqrt{(x_2-x_1)^2+(y_2-y_1)^2+(z_2-z_1)^2} \\
\end{align}

In [279]:
import csv
import codecs
import numpy as np
import xml.etree.ElementTree

np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

## Load Video generate Coordinates

In [280]:
fname = "data/3d_gen_avi.tsv"
coordinates = []
with codecs.open(fname, mode='rb', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')
    headers = reader.next()
    ix1, iy1, iy2 = ( headers.index('x1'), headers.index('y1'), headers.index('y2') )
    for row in reader:        
        coordinates.append( [float(row[ix1]),float(row[iy1]),float(row[iy2])] )
ref_coordinates = np.array(coordinates)[1:] # Jump B-VOPs AVI

## Load VpCore2 PHI File

In [281]:
filen = "data/001.phi"
e = xml.etree.ElementTree.parse(filen).getroot()
phi_data = []
for elm in e.findall('Data'):
    status,x,y,z = elm.get('algostatus'), float(elm.get('x')), float(elm.get('y')), float(elm.get('z'))
    if status != 'errornoobject':
        phi_data.append( [x,y,z] )
phi_data = np.array(phi_data)

## Difference from reference coordinate

In [282]:
print "Difference from reference coordinate = ", np.sum( phi_data - ref_coordinates )

Difference from reference coordinate =  0.0


## Calculation Total Distance

In [283]:
total_diff = np.sqrt( np.sum( np.diff(arr, axis=0)**2, axis=1 ) )
total_distance = np.sum( total_diff )
print("PHI total distance = %f" % total_distance )

PHI total distance = 7535.578343


## Distance from VpCore2 results file

Two first values from are tag unitialize, that explain difference from PHI data.

In [284]:
filen = "data/001.phr"
e = xml.etree.ElementTree.parse(filen).getroot()
sumdist = 0
for elm in e.findall('*/Data'):
    if elm.get('datatype') == 'Locomotion3D':
        inadist,smldist,lardist = ( float(elm.get('inadist')), float(elm.get('smldist')), float(elm.get('lardist')) )
        sumdist += inadist + smldist + lardist
print "Results total distance =", sumdist
print "Diff =", (total_distance-sumdist), ", Two first data =", np.sum(total_diff[:2])

Results total distance = 7488.684
Diff = 46.8943426632 , Two first data = 46.8943662193


## Check Replay RawData

In [285]:
filen = "Data/001Replay.phi"
e = xml.etree.ElementTree.parse(filen).getroot()
phi_replay_data = []
for elm in e.findall('Data'):
    status,x,y,z = elm.get('algostatus'), float(elm.get('x')), float(elm.get('y')), float(elm.get('z'))
    if status != 'errornoobject':
        phi_replay_data.append( [x,y,z] )
phi_replay_data = np.array(phi_replay_data)
print "Difference from reference coordinate = ", np.sum( phi_data - ref_coordinates )

Difference from reference coordinate =  0.0
